In [1]:
import warnings
warnings.filterwarnings("ignore")

# Inicialización

Importamos las librerías que nos servirán para poder procesar, visualizar y explorar nuestros datos de manera efectiva, la librería `pandas`, `request`.

In [2]:
import pandas as pd
import requests

# Extracción de los datos desde la API


Hemos realizado un proceso que nos permite acceder a los datos proporcionados por la API en línea. Aquí está un resumen de los pasos que seguimos para obtener estos datos:

- **Acceso a la API:** En primer lugar, identificamos que había una fuente de datos en línea proporcionada por la página web sugerida. Esta fuente de datos estaba disponible a través de una API, lo que significa que podíamos obtener datos estructurados de manera programática en lugar de navegar por la página web manualmente.

- **Solicitud de Clave de Autenticación:** Para acceder a la API, requerimos una clave de autenticación. Hemos solicitado y obtenido una clave de autenticación, que se incluyó en la URL de la solicitud.

- **Solicitud de Datos:** Usando la biblioteca requests en Python, realizamos una solicitud GET a la URL de la API que contiene la clave de autenticación. Esto nos permitió acceder a los datos de la API.

- **Manejo de la Respuesta:** Verificamos si la solicitud fue exitosa comprobando el código de estado de la respuesta. En caso de una respuesta exitosa (código de estado 200), extraemos los datos en formato JSON de la respuesta y los almacenamos en una variable llamada data.

En resumen, este código nos permitió acceder a los datos de la API proporcionada por la página web, asegurándonos de que nuestra solicitud estuviera autorizada mediante la clave de autenticación.

In [3]:
# URL de la API con API_KEY
url = 'http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/'
key = {'auth_key': 'ypPRK6EXIi57RaKGk1H9au9WCQyOpUj0fHZQ4T2v'}


# solicitud GET a la API
response = requests.get(url,params=key)

# Verifica si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Extrae los datos JSON de la respuesta
    data = response.json()
    # Ahora 'data' contiene los datos en formato JSON
    print('Solicitud completada, lista obtenida')
else:
    print("Error al hacer la solicitud. Código de estado:", response.status_code)


Solicitud completada, lista obtenida


In [4]:
# Observar el primer elemento de la lista
data[0]

{'status': None,
 'description': 'Listado de prestadores con licencias TIC activas',
 'parameters': [],
 'tags': [],
 'last_revision_id': None,
 'timestamp': 1699663378917,
 'created_at': 1654220805,
 'title': 'Listado de licencias TIC otorgadas',
 'modified_at': 1654220996,
 'category_id': '83742',
 'methods': None,
 'sources': [],
 'total_revisions': None,
 'frequency': 'monthly',
 'link': 'http://datosabiertos.enacom.gob.ar/dataviews/259940/listado-de-licencias-tic-otorgadas/',
 'user': 'asrozada',
 'status_str': None,
 'guid': 'LISTA-DE-LICEN-TIC-OTORG',
 'category_name': 'Licencias',
 'endpoint': 'https://www.enacom.gob.ar/public/datosabiertos/Licencias_tics/LicenciasTics.csv'}

In [5]:
# Contar cuántos elementos hay en la lista
len(data)

97

Al examinar y explorar el conjunto de datos denominado `data`, se revela que este consta de un total de 97 conjuntos de datos diversos. Para cumplir con nuestros objetivos, hemos optado por realizar un filtrado inicial basado en la variable 'category_name'. Este filtro nos permitirá centrar nuestra atención en la categoría de datos relacionada con 'Acceso a Internet'. Sin embargo, es importante destacar que, a medida que avancemos en nuestro proyecto, evaluaremos de manera continua la posibilidad de incluir otros conjuntos de datos según las necesidades específicas que vayan surgiendo.

La variable dataset_list contendrá solo los elementos de datos que pertenecen a esta categoría.

In [6]:
dataset_list = [item for item in data if item['category_name'] == 'Acceso a Internet']

# Recopilación y Procesamiento de Metadata de Conjuntos de Datos

## Introducción

En esta sección, abordamos a mayor profundidad la tarea de recopilar y procesar metadata de conjuntos de datos relacionados con 'Acceso a Internet' al rededor de toda la API. Nuestra meta es utilizar la documentación de la API v2 para acceder a estos conjuntos de datos y descargar sus contenidos de manera eficiente.

## Conjunto de Datos

Podemos consumir la metadata de los recursos de tipo conjunto de datos (dataset) publicados en Junar a través de una simple llamada GET. Las llamadas GET nos pueden traer una lista completa de todos los conjuntos de datos o solo de un GUID específico. Las llamadas GET nos devuelven los siguientes parámetros:

- GET /api/v2/datasets.json
- GET /api/v2/datasets/{guid}.json

Estos parámetros son esenciales para llevar a cabo esta operación de manera exitosa y recorrer toda la API, así como el siguiente diccionario proporcionado por la documentación de la API:

- **guid**: Identificador del recurso
- **title**: Título del conjunto de datos
- **description**: Descripción del conjunto de datos
- **categories**: Nombre de la categoría
- **endpoint**: URL apuntando al recurso con los datos (archivos o página web)
- **tags**: Opcionales. Tags separados por coma.
- **user**: Usuario que publica el recurso.
- **parameters**: Parámetros que tiene el recurso.
- **created_at**: Fecha de creación de la versión del recurso
- **link**: Link a la vista del recurso en el portal


Comenzamos filtrando la metadata para identificar conjuntos de datos relevantes y luego recorremos estos conjuntos. Utilizamos un identificador único `GUID` para construir las `URL` necesarias y realizar solicitudes `GET` con el fin de descargar los datos en formato CSV. Cuando las solicitudes son exitosas, creamos DataFrames que procesan la información descargada y la guardan en archivos CSV. Además, proporcionamos un informe de progreso detallado para cada conjunto de datos y gestionamos errores en caso de que no se pueda obtener la información.


In [8]:
# Itera a través de los conjuntos de datos
for i, item in enumerate(dataset_list):
    # Obtiene el GUID del conjunto de datos
    guid = item['guid']
    # Obtiene el título del conjunto de datos y realiza ajustes en el formato
    title = item['title'].lower().replace(' ', '_')
    # Construye la URL completa para obtener el conjunto de datos por su GUID
    dataset_url = f"{url}{guid}/data.csv/?auth_key={key['auth_key']}"
    # Realiza una solicitud GET para obtener la información del conjunto de datos
    dataset_response = requests.get(dataset_url)
    
    # Comprueba si la solicitud fue exitosa
    if dataset_response.status_code == 200:
        # Crea un DataFrame a partir de los datos
        df = pd.read_csv(dataset_url)

        # Guarda el DataFrame en un archivo CSV
        csv_filename = f'{title}.csv'
        df.to_csv('./datasets/' + csv_filename, index=False)

        print(f'Dataset {i + 1}:')
        print(f'Título del conjunto de datos: {item["title"]}')
        print(f'Nombre del archivo CSV: {csv_filename}')
        print('---')

    else:
        print(f'No se pudo obtener la información del conjunto de datos con GUID {guid}')


Dataset 1:
Título del conjunto de datos: Penetración del Internet fijo por provincia (accesos por cada 100 habitantes)
Nombre del archivo CSV: penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_habitantes).csv
---
Dataset 2:
Título del conjunto de datos: Penetración nacional del Internet fijo
Nombre del archivo CSV: penetración_nacional_del_internet_fijo.csv
---
Dataset 3:
Título del conjunto de datos: Penetración del Internet fijo por provincia (accesos por cada 100 hogares)
Nombre del archivo CSV: penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_hogares).csv
---
Dataset 4:
Título del conjunto de datos: Acceso a Internet Fijo por rangos de velocidad de bajada y provincia
Nombre del archivo CSV: acceso_a_internet_fijo_por_rangos_de_velocidad_de_bajada_y_provincia.csv
---
Dataset 5:
Título del conjunto de datos: Total nacional de accesos a Internet fijo por velocidad de bajada
Nombre del archivo CSV: total_nacional_de_accesos_a_internet_fijo_por_velocidad

Exploramos el proceso de recopilar y procesar metadata de conjuntos de datos relacionados con 'Acceso a Internet' utilizando la documentación de la API v2 proporcionada por Junar. Este enfoque nos permitió automatizar la obtención de la información requerida; como resultado, conseguimos obtener un conjunto de 18 datasets que contienen información relevante sobre 'Acceso a Internet'. 

Continuamos con el proceso de ETL (Extracción, Transformación y Carga). Nuestro objetivo es asegurarnos de que los datos estén en las mejores condiciones posibles para su análisis posterior. Esto incluirá la eliminación de valores nulos que puedan afectar la integridad de los datos, la identificación y manejo de posibles valores duplicados, y la conversión de datos a formatos numéricos cuando sea necesario. Además, llevaremos a cabo diversas transformaciones y procesamientos, como el reemplazo de comas por puntos, con el fin de garantizar que los datos estén limpios y listos para su análisis detallado. Asimismo, exploraremos la combinación de dataframes con el propósito de reducir la complejidad de los datos y consolidar conjuntos de datos relacionados en uno solo. Esta estrategia busca optimizar el proceso de análisis al reducir el número de dataframes y facilitar la interpretación de los datos.

Comenzaremos cargando los datos en dataframes para observar la estructura de algunas variables y tener una primera vista general de los mismos.

# **Dataset 1:**  
# **Penetración del Internet fijo por provincia (accesos por cada 100 habitantes)**  
Nombre del archivo CSV: penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_habitantes).csv

In [9]:
# carga del archivo:
accesos_por_cada_100_habitantes = pd.read_csv('./datasets/penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_habitantes).csv')

In [10]:
accesos_por_cada_100_habitantes.shape

(864, 4)

In [11]:
accesos_por_cada_100_habitantes.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hab
0,2022,4,Buenos Aires,"26,72"
1,2022,4,Capital Federal,"50,35"
2,2022,4,Catamarca,"16,73"
3,2022,4,Chaco,"11,66"
4,2022,4,Chubut,"26,23"


In [12]:
accesos_por_cada_100_habitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Año                       864 non-null    int64 
 1   Trimestre                 864 non-null    int64 
 2   Provincia                 864 non-null    object
 3   Accesos por cada 100 hab  864 non-null    object
dtypes: int64(2), object(2)
memory usage: 27.1+ KB


In [13]:
accesos_por_cada_100_habitantes['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [14]:
accesos_por_cada_100_habitantes['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

In [15]:
accesos_por_cada_100_habitantes['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [16]:
accesos_por_cada_100_habitantes.Provincia.nunique()

24

conversión de la columna "Accesos por cada 100 hab" al formato numérico para facilitar análisis estadísticos y visualizaciones más precisas.

In [17]:
accesos_por_cada_100_habitantes['Accesos por cada 100 hab'] = pd.to_numeric(accesos_por_cada_100_habitantes['Accesos por cada 100 hab'].str.replace(',', '.'), errors='coerce')

In [18]:
accesos_por_cada_100_habitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Año                       864 non-null    int64  
 1   Trimestre                 864 non-null    int64  
 2   Provincia                 864 non-null    object 
 3   Accesos por cada 100 hab  864 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.1+ KB


El conjunto de datos, que abarca un periodo de 2014 a 2022, se ha revisado detalladamente para confirmar la presencia de información coherente en cuanto a años, trimestres y provincias. Se ha observado que los registros abarcan los años 2014 hasta 2022 y los cuatro trimestres de cada año, con un total de 24 provincias. Como parte del proceso de mejora, hemos convertido la columna 'Accesos por cada 100 hab' al formato numérico (float) para garantizar que la información sea sólida y completa, facilitando análisis estadísticos y visualizaciones más precisas.

Después de haber estudiado y transformado las variables que lo necesitaban,
procedemos a guardar el DataFrame en un archivo CSV.




In [19]:
# Guardar el DataFrame en un archivo CSV
accesos_por_cada_100_habitantes.to_csv('./datasets_cleaned/accesos_por_cada_100_habitantes.csv', index=False)


# **Dataset 2:**  
# **Penetración nacional del Internet fija**   
Nombre del archivo CSV: penetración_nacional_del_internet_fijo.csv

In [20]:
penetración_nacional_del_internet_fijo = pd.read_csv('./datasets/penetración_nacional_del_internet_fijo.csv')

In [21]:
penetración_nacional_del_internet_fijo.shape

(36, 5)

In [24]:
penetración_nacional_del_internet_fijo.head()

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2022,4,"77,21","24,15",Oct-Dic 2022
1,2022,3,"76,64","23,95",Jul-Sept 2022
2,2022,2,"75,97","23,72",Abr-Jun 2022
3,2022,1,"73,88","23,05",Ene-Mar 2022
4,2021,4,"73,18","22,81",Oct-Dic 2021


In [25]:
penetración_nacional_del_internet_fijo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Año                           36 non-null     int64 
 1   Trimestre                     36 non-null     int64 
 2   Accesos por cada 100 hogares  36 non-null     object
 3   Accesos por cada 100 hab      36 non-null     object
 4   Periodo                       36 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.5+ KB


Para el análisis, se optará por trabajar exclusivamente con el Dataset 1 ("Penetración del Internet fijo por provincia - accesos por cada 100 habitantes") y el Dataset 3 ("Penetración del Internet fijo por provincia - accesos por cada 100 hogares"). Esta decisión se basa en la premisa de que mediante la combinación y análisis de estos dos conjuntos de datos, se podrá obtener la información que nos proporciona este Dataset, por lo tanto no lo usaremos.

# **Dataset 3:**  
# **Penetración del Internet fijo por provincia (accesos por cada 100 hogares)**    
Nombre del archivo CSV: penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_hogares).csv

In [26]:
# carga del archivo:
accesos_por_cada_100_hogares = pd.read_csv('./datasets/penetración_del_internet_fijo_por_provincia_(accesos_por_cada_100_hogares).csv')

In [27]:
accesos_por_cada_100_hogares.shape

(864, 7)

In [28]:
accesos_por_cada_100_hogares.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,2022,4,Buenos Aires,"78,94",NaN,NaN,NaN
1,2022,4,Capital Federal,"122,73",NaN,NaN,NaN
2,2022,4,Catamarca,"65,77",NaN,NaN,NaN
3,2022,4,Chaco,"43,62",NaN,NaN,NaN
4,2022,4,Chubut,"85,57",NaN,NaN,NaN


In [29]:
accesos_por_cada_100_hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           864 non-null    int64  
 1   Trimestre                     864 non-null    int64  
 2   Provincia                     864 non-null    object 
 3   Accesos por cada 100 hogares  864 non-null    object 
 4   Unnamed: 4                    0 non-null      float64
 5   Unnamed: 5                    0 non-null      float64
 6   Unnamed: 6                    0 non-null      float64
dtypes: float64(3), int64(2), object(2)
memory usage: 47.4+ KB


In [30]:
accesos_por_cada_100_hogares['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [31]:
accesos_por_cada_100_hogares['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

In [32]:
accesos_por_cada_100_hogares['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [33]:
accesos_por_cada_100_hogares.Provincia.nunique()

24

Observamos que las últimas tres columnas de nuestro conjunto de datos están compuestas únicamente por valores nulos (NaN), hemos procedido a eliminarlas para optimizar la estructura del conjunto y facilitar futuros análisis.

In [34]:
accesos_por_cada_100_hogares = accesos_por_cada_100_hogares.iloc[:, :4]

In [35]:
accesos_por_cada_100_hogares.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,4,Buenos Aires,"78,94"
1,2022,4,Capital Federal,"122,73"
2,2022,4,Catamarca,"65,77"
3,2022,4,Chaco,"43,62"
4,2022,4,Chubut,"85,57"


In [36]:
accesos_por_cada_100_hogares['Accesos por cada 100 hogares'] = pd.to_numeric(accesos_por_cada_100_hogares['Accesos por cada 100 hogares'].str.replace(',', '.'), errors='coerce')

In [37]:
accesos_por_cada_100_hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           864 non-null    int64  
 1   Trimestre                     864 non-null    int64  
 2   Provincia                     864 non-null    object 
 3   Accesos por cada 100 hogares  864 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.1+ KB


El conjunto de datos ofrece una visión detallada de la cantidad de accesos a Internet fijo por cada 100 hogares en diversas provincias de Argentina a lo largo de varios años y trimestres. Con un total de 864 entradas y originalmente 7 columnas, que incluyen el año, trimestre, provincia y la medida de accesos por cada 100 hogares, se observó que las últimas tres columnas se componían únicamente de valores nulos (NaN). Por ende, se procedió a eliminar estas columnas para optimizar la estructura del conjunto y facilitar futuros análisis. Tras la revisión y optimización, la columna "Accesos por cada 100 hogares" se convirtió al formato numérico (float) para asegurar que la información sea sólida y completa. 

El conjunto de datos resultante ahora consta de 864 entradas y 4 columnas, brindando una perspectiva más clara y eficiente de la penetración del Internet fijo en hogares a nivel provincial.

Después de haber estudiado y transformado las variables que lo necesitaban, procedemos a guardar el DataFrame en un archivo CSV.

In [38]:
# Guardar el DataFrame en un archivo CSV
accesos_por_cada_100_hogares.to_csv('datasets_cleaned/accesos_por_cada_100_hogares.csv', index=False)

# **Dataset 4:**  
# **Acceso a Internet Fijo por rangos de velocidad de bajada y provincia**  
Nombre del archivo CSV: acceso_a_internet_fijo_por_rangos_de_velocidad_de_bajada_y_provincia.csv

In [39]:
# carga del archivo:
acceso_rangos_velocidad_de_bajada_y_provincia = pd.read_csv('./datasets/acceso_a_internet_fijo_por_rangos_de_velocidad_de_bajada_y_provincia.csv')

In [40]:
acceso_rangos_velocidad_de_bajada_y_provincia.shape

(864, 12)

In [41]:
acceso_rangos_velocidad_de_bajada_y_provincia

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,97.634,4.794.665
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,106.772,1.551.926
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739,2.280,70.909
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866,3.909,143.663
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048,20.026,168.759
...,...,...,...,...,...,...,...,...,...,...,...,...
859,2014,1,Santa Cruz,161.000,1.625,24.972,1.000,1.000,0.000,0,0.000,26.760
860,2014,1,Santa Fe,8.456,124.468,345.225,20.328,6.845,23.000,668,0.000,506.013
861,2014,1,Santiago Del Estero,1.234,10.531,22.817,2.422,109.000,0.000,0,0.000,37.113
862,2014,1,Tierra Del Fuego,12.000,607.000,30.902,6.000,0.000,0.000,0,0.000,31.527


In [42]:
acceso_rangos_velocidad_de_bajada_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Año                  864 non-null    int64  
 1   Trimestre            864 non-null    int64  
 2   Provincia            864 non-null    object 
 3   HASTA 512 kbps       864 non-null    float64
 4   + 512 Kbps - 1 Mbps  864 non-null    float64
 5   + 1 Mbps - 6 Mbps    864 non-null    object 
 6   + 6 Mbps - 10 Mbps   864 non-null    float64
 7   + 10 Mbps - 20 Mbps  864 non-null    float64
 8   + 20 Mbps - 30 Mbps  864 non-null    float64
 9   + 30 Mbps            864 non-null    object 
 10  OTROS                858 non-null    float64
 11  Total                864 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 81.1+ KB


En la columna 'OTROS' de nuestro conjunto de datos, hemos identificado la presencia de 6 valores nulos. 

In [43]:
acceso_rangos_velocidad_de_bajada_y_provincia[acceso_rangos_velocidad_de_bajada_y_provincia['OTROS'].isnull()]

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
337,2019,2,Capital Federal,74.0,1.027,281.370,124.087,239.691,296.155,475.991,NaN,1.418.395
347,2019,2,La Rioja,9.0,11.000,39.819,4.297,4.857,13.000,1.298,NaN,50.304
348,2019,2,Mendoza,20.0,2.522,205.165,2.971,10.092,1.256,234,NaN,222.260
355,2019,2,Santa Cruz,25.0,84.000,24.861,3.806,1.202,28.000,4,NaN,30.010
358,2019,2,Tierra Del Fuego,6.0,123.000,41.824,4.241,1.573,96.000,32,NaN,47.895
359,2019,2,Tucumán,16.0,157.000,78.891,44.296,24.291,628.000,18.394,NaN,166.673


Cambiamos los valores nulos en la columna 'OTROS' por ceros, utilizando el método fillna()

In [44]:
acceso_rangos_velocidad_de_bajada_y_provincia['OTROS'].fillna(0, inplace=True)

In [45]:
valores_unicos = acceso_rangos_velocidad_de_bajada_y_provincia['OTROS'].unique()
valores_unicos.sort()
print(valores_unicos)

[ -1.945  -1.      0.      1.      1.035   1.048   1.123   1.144   1.246
   1.305   1.313   1.483   1.492   1.566   1.617   1.618   1.68    1.717
   1.737   1.902   1.942   1.967   2.      2.003   2.151   2.154   2.196
   2.206   2.207   2.208   2.233   2.25    2.264   2.28    2.309   2.324
   2.378   2.401   2.421   2.424   2.442   2.45    2.453   2.465   2.52
   2.57    2.58    2.635   2.637   2.661   2.68    2.811   2.833   3.
   3.012   3.027   3.065   3.094   3.098   3.176   3.287   3.351   3.353
   3.503   3.708   3.711   3.719   3.909   3.932   3.94    3.942   3.975
   3.999   4.029   4.062   4.121   4.144   4.15    4.152   4.267   4.39
   4.402   4.43    4.435   4.5     4.501   4.516   4.588   4.641   4.771
   4.779   4.78    4.801   4.985   4.995   5.01    5.017   5.02    5.053
   5.068   5.172   5.453   5.579   5.645   5.72    5.758   6.08    6.105
   6.119   6.287   6.319   6.327   6.328   6.369   6.441   6.475   6.489
   6.49    6.491   6.494   6.5     6.531   6.69    6.747

No tiene sentido tener valores negativos para un rango de velocidad de Internet. Los valores negativos en esta columna podrían ser el resultado de algún error o problema en la recopilación de datos.

Eliminamos estos datos inconsistentes

In [46]:
acceso_rangos_velocidad_de_bajada_y_provincia = acceso_rangos_velocidad_de_bajada_y_provincia[acceso_rangos_velocidad_de_bajada_y_provincia['OTROS'] >= 0]


Revisamos los valores únicos presentes en las columnas `'Año'`, `'Trimestre'` y `'Provincia'`

In [47]:
acceso_rangos_velocidad_de_bajada_y_provincia['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [48]:
acceso_rangos_velocidad_de_bajada_y_provincia['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

In [49]:
acceso_rangos_velocidad_de_bajada_y_provincia['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [50]:
acceso_rangos_velocidad_de_bajada_y_provincia.Provincia.nunique()

24

Convertir las columnas de rango de velocidad de bajada a formato entero 

In [51]:
# Lista de columnas a convertir:
columnas_a_convertir = ['HASTA 512 kbps', '+ 512 Kbps - 1 Mbps', '+ 1 Mbps - 6 Mbps', '+ 6 Mbps - 10 Mbps', '+ 10 Mbps - 20 Mbps', '+ 20 Mbps - 30 Mbps', '+ 30 Mbps', 'OTROS', 'Total']

In [52]:
acceso_rangos_velocidad_de_bajada_y_provincia[columnas_a_convertir] = acceso_rangos_velocidad_de_bajada_y_provincia[columnas_a_convertir].applymap( lambda x: str(x).replace('.', ''))

In [53]:
acceso_rangos_velocidad_de_bajada_y_provincia[columnas_a_convertir] = acceso_rangos_velocidad_de_bajada_y_provincia[columnas_a_convertir].astype(int)

In [54]:
acceso_rangos_velocidad_de_bajada_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 862 entries, 0 to 863
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Año                  862 non-null    int64 
 1   Trimestre            862 non-null    int64 
 2   Provincia            862 non-null    object
 3   HASTA 512 kbps       862 non-null    int32 
 4   + 512 Kbps - 1 Mbps  862 non-null    int32 
 5   + 1 Mbps - 6 Mbps    862 non-null    int32 
 6   + 6 Mbps - 10 Mbps   862 non-null    int32 
 7   + 10 Mbps - 20 Mbps  862 non-null    int32 
 8   + 20 Mbps - 30 Mbps  862 non-null    int32 
 9   + 30 Mbps            862 non-null    int32 
 10  OTROS                862 non-null    int32 
 11  Total                862 non-null    int32 
dtypes: int32(9), int64(2), object(1)
memory usage: 57.2+ KB


Revisamos los valores minimos y maximos de cada una de las variables (rangos de velocidad de bajada)

In [55]:
acceso_rangos_velocidad_de_bajada_y_provincia['+ 30 Mbps'].sum()

73916013

Enfrentamos la problemática de valores nulos abordando específicamente los 6 presentes en la columna 'OTROS', sustituyéndolos por ceros. Asimismo, eliminamos valores negativos en dicha columna que pudieran derivar de errores en la recopilación de datos. Verificamos la coherencia en las columnas clave 'Año', 'Trimestre' y 'Provincia'. Además, llevamos a cabo la conversión de las columnas de velocidad de bajada al formato entero para una mayor uniformidad.

A pesar de estos esfuerzos, persisten algunas inconsistencias en los datos. En vista de esto, estamos considerando la opción de extraer directamente la información desde la página web, permitiéndonos así realizar una comparación exhaustiva con el conjunto de datos actual. 

Procedemos a extraer los datos descargados

In [56]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel = pd.read_excel('./datasets_descargados/-1Internet_Accesos-por-velocidad.xlsx',sheet_name='Accesos por velocidad')

In [226]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel.head()

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,4,Buenos Aires,28934.0,23347.0,259866.0,281550.0,244555.0,98670.0,3760109.0,97634.0,4794665.0
1,2022,4,Capital Federal,516.0,5096.0,31320.0,64653.0,36336.0,8116.0,1299117.0,106772.0,1551926.0
2,2022,4,Catamarca,71.0,315.0,2716.0,5028.0,4638.0,3122.0,52739.0,2280.0,70909.0
3,2022,4,Chaco,461.0,771.0,16067.0,17846.0,6704.0,3039.0,94866.0,3909.0,143663.0
4,2022,4,Chubut,109.0,1614.0,46457.0,31007.0,34728.0,15770.0,19048.0,20026.0,168759.0


In [224]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Año                  864 non-null    int64  
 1   Trimestre            864 non-null    int64  
 2   Provincia            864 non-null    object 
 3   HASTA 512 kbps       864 non-null    float64
 4   + 512 Kbps - 1 Mbps  864 non-null    float64
 5   + 1 Mbps - 6 Mbps    864 non-null    float64
 6   + 6 Mbps - 10 Mbps   864 non-null    float64
 7   + 10 Mbps - 20 Mbps  864 non-null    float64
 8   + 20 Mbps - 30 Mbps  864 non-null    float64
 9   + 30 Mbps            864 non-null    float64
 10  OTROS                858 non-null    float64
 11  Total                864 non-null    float64
dtypes: float64(9), int64(2), object(1)
memory usage: 81.1+ KB


Repetimos lo que hicimos con la data anterior, revisar valores nulos, revisar duplicados y algunos datas atipicos.

In [231]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel[acceso_rangos_velocidad_de_bajada_y_provincia_excel['OTROS'].isnull()]

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
337,2019,2,Capital Federal,74.0,1027.0,281370.0,124087.0,239691.0,296155.0,475991.0,NaN,1418395.0
347,2019,2,La Rioja,9.0,11.0,39819.0,4297.0,4857.0,13.0,1298.0,NaN,50304.0
348,2019,2,Mendoza,20.0,2522.0,205165.0,2971.0,10092.0,1256.0,234.0,NaN,222260.0
355,2019,2,Santa Cruz,25.0,84.0,24861.0,3806.0,1202.0,28.0,4.0,NaN,30010.0
358,2019,2,Tierra Del Fuego,6.0,123.0,41824.0,4241.0,1573.0,96.0,32.0,NaN,47895.0
359,2019,2,Tucumán,16.0,157.0,78891.0,44296.0,24291.0,628.0,18394.0,NaN,166673.0


In [233]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel['OTROS'].fillna(0, inplace=True)

In [236]:
acceso_rangos_velocidad_de_bajada_y_provincia_excel = acceso_rangos_velocidad_de_bajada_y_provincia_excel[acceso_rangos_velocidad_de_bajada_y_provincia_excel['OTROS'] >= 0]

Tomamos la decision de trabajar con esta data descargada desde la pagina web, despues de haber hecho algunas modificaciones en el proceso, vamos a guardar en archivo csv en la carpeta datasets_cleaned

In [237]:
# Guardar el DataFrame en un archivo CSV
acceso_rangos_velocidad_de_bajada_y_provincia_excel.to_csv('datasets_cleaned/acceso_rangos_velocidad_de_bajada_y_provincia.csv', index=False)

# **Dataset 5:**  
# **Total nacional de accesos a Internet fijo por velocidad de bajada**    
Nombre del archivo CSV: total_nacional_de_accesos_a_internet_fijo_por_velocidad_de_bajada.csv

In [60]:
total_nacional_velocidad_de_bajada = pd.read_csv('./datasets/total_nacional_de_accesos_a_internet_fijo_por_velocidad_de_bajada.csv')

In [61]:
total_nacional_velocidad_de_bajada.shape

(36, 11)

In [62]:
total_nacional_velocidad_de_bajada.head()

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total
0,2022,4,31.981,85.668,1.075.272,1.109.421,765.015,335.573,7.410.902,394.282,11.208.114
1,2022,3,33.013,96.727,1.161.370,1.151.906,823.505,479.822,6.995.750,349.035,11.091.128
2,2022,2,33.667,99.498,1.193.090,1.197.030,856.562,485.321,6.741.922,351.594,10.958.684
3,2022,1,34.890,104.840,1.263.273,1.209.148,967.508,509.830,6.336.187,198.333,10.624.009
4,2021,4,41.262,28.521,1.413.208,1.245.333,976.539,558.358,6.032.322,194.251,10.489.794


In [63]:
total_nacional_velocidad_de_bajada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Año                      36 non-null     int64  
 1   Trimestre                36 non-null     int64  
 2   Hasta 512 kbps           36 non-null     float64
 3   Entre 512 Kbps y 1 Mbps  36 non-null     float64
 4   Entre 1 Mbps y 6 Mbps    36 non-null     object 
 5   Entre 6 Mbps y 10 Mbps   36 non-null     object 
 6   Entre 10 Mbps y 20 Mbps  36 non-null     object 
 7   Entre 20 Mbps y 30 Mbps  36 non-null     object 
 8   Más de 30 Mbps           36 non-null     object 
 9   OTROS                    36 non-null     float64
 10  Total                    36 non-null     object 
dtypes: float64(3), int64(2), object(6)
memory usage: 3.2+ KB


Llevamos a cabo una transformación de los valores mediante el uso de la función applymap, donde reemplazamos los puntos decimales por una cadena vacía para considerar los miles, y posteriormente realizamos la conversión de los datos a formato entero.

In [64]:
total_nacional_velocidad_de_bajada = total_nacional_velocidad_de_bajada.applymap(lambda x: int( str(x).replace('.', '') ) )

In [65]:
total_nacional_velocidad_de_bajada.head()

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total
0,2022,4,31981,85668,1075272,1109421,765015,335573,7410902,394282,11208114
1,2022,3,33013,96727,1161370,1151906,823505,479822,6995750,349035,11091128
2,2022,2,33667,99498,1193090,1197030,856562,485321,6741922,351594,10958684
3,2022,1,3489,10484,1263273,1209148,967508,509830,6336187,198333,10624009
4,2021,4,41262,28521,1413208,1245333,976539,558358,6032322,194251,10489794


In [66]:
total_nacional_velocidad_de_bajada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Año                      36 non-null     int64
 1   Trimestre                36 non-null     int64
 2   Hasta 512 kbps           36 non-null     int64
 3   Entre 512 Kbps y 1 Mbps  36 non-null     int64
 4   Entre 1 Mbps y 6 Mbps    36 non-null     int64
 5   Entre 6 Mbps y 10 Mbps   36 non-null     int64
 6   Entre 10 Mbps y 20 Mbps  36 non-null     int64
 7   Entre 20 Mbps y 30 Mbps  36 non-null     int64
 8   Más de 30 Mbps           36 non-null     int64
 9   OTROS                    36 non-null     int64
 10  Total                    36 non-null     int64
dtypes: int64(11)
memory usage: 3.2 KB


Al realizar esta acción, facilitamos futuras operaciones y cálculos, garantizando la integridad de los datos para un análisis más preciso y efectivo.

Después de haber estudiado y transformado las variables que lo necesitaban, procedemos a guardar el DataFrame en un archivo CSV.

In [67]:
# Guardar el DataFrame en un archivo CSV
total_nacional_velocidad_de_bajada.to_csv('datasets_cleaned/total_nacional_velocidad_de_bajada.csv', index=False)

# **Dataset 6:**  
# **Velocidad media de bajada de Internet fijo por provincia**    
Nombre del archivo CSV: velocidad_media_de_bajada_de_internet_fijo_por_provincia.csv

In [68]:
velocidad_media_de_bajada_por_provincia = pd.read_csv('./datasets/velocidad_media_de_bajada_de_internet_fijo_por_provincia.csv')

In [69]:
velocidad_media_de_bajada_por_provincia.shape

(864, 4)

In [70]:
velocidad_media_de_bajada_por_provincia.head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,4,Buenos Aires,111.000
1,2022,4,Capital Federal,18.286
2,2022,4,Catamarca,8.333
3,2022,4,Chaco,10.402
4,2022,4,Chubut,1.592


In [71]:
velocidad_media_de_bajada_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     864 non-null    int64  
 1   Trimestre               864 non-null    int64  
 2   Provincia               864 non-null    object 
 3   Mbps (Media de bajada)  864 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 27.1+ KB


In [72]:
velocidad_media_de_bajada_por_provincia['Mbps (Media de bajada)'] = velocidad_media_de_bajada_por_provincia['Mbps (Media de bajada)'].apply(lambda x: float(str(x).replace(',', '.')))

Ordenamos los valores únicos de la columna 'Mbps (Media de bajada)' de manera ascendente. Esto nos permite visualizar las velocidades de bajada de Internet de menor a mayor en el conjunto de datos.

In [73]:
velocidades_unicas = velocidad_media_de_bajada_por_provincia['Mbps (Media de bajada)'].unique()
velocidades_unicas.sort()

print(velocidades_unicas)


[  1.      1.006   1.011   1.019   1.03    1.038   1.041   1.046   1.049
   1.052   1.064   1.066   1.07    1.072   1.073   1.084   1.088   1.089
   1.097   1.101   1.102   1.105   1.106   1.109   1.112   1.117   1.128
   1.132   1.149   1.158   1.163   1.164   1.167   1.17    1.175   1.176
   1.181   1.196   1.197   1.199   1.205   1.207   1.208   1.214   1.219
   1.228   1.236   1.244   1.249   1.25    1.251   1.253   1.255   1.26
   1.27    1.272   1.279   1.289   1.296   1.297   1.299   1.318   1.322
   1.335   1.337   1.34    1.344   1.347   1.348   1.349   1.351   1.383
   1.384   1.385   1.388   1.414   1.42    1.433   1.444   1.45    1.453
   1.467   1.468   1.47    1.475   1.482   1.484   1.491   1.507   1.52
   1.526   1.557   1.57    1.573   1.581   1.584   1.586   1.588   1.589
   1.592   1.598   1.601   1.607   1.634   1.639   1.657   1.658   1.67
   1.678   1.687   1.693   1.701   1.718   1.724   1.769   1.775   1.793
   1.81    1.817   1.828   1.843   1.853   1.894   1.9

Al explorar el conjunto de datos sobre la velocidad media de bajada de Internet fijo, llevamos a cabo una serie de procesos importantes para garantizar la calidad y pero no encontramos la coherencia de la información. 

Decidimos extraer los datos directamente desde la página web como una alternativa y dar una primera ojeada.

In [74]:
velocidad_media_de_bajada_por_provincia_excel = pd.read_excel('./datasets_descargados/-14historico_velocidad_internet.xlsx')

In [75]:
velocidad_media_de_bajada_por_provincia_excel.head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,4,Buenos Aires,111.00
1,2022,4,Capital Federal,182.86
2,2022,4,Catamarca,83.33
3,2022,4,Chaco,104.02
4,2022,4,Chubut,15.92


In [76]:
# Obtener el mínimo y máximo de la columna 'Mbps (Media de bajada)'
minimo_valor = velocidad_media_de_bajada_por_provincia_excel['Mbps (Media de bajada)'].min()
maximo_valor = velocidad_media_de_bajada_por_provincia_excel['Mbps (Media de bajada)'].max()

# Verificar si hay valores nulos en la columna 'Mbps (Media de bajada)'
hay_nulos = velocidad_media_de_bajada_por_provincia_excel['Mbps (Media de bajada)'].isnull().any()

# Imprimir los resultados
print(f'Mínimo valor: {minimo_valor}')
print(f'Máximo valor: {maximo_valor}')
print(f'¿Hay valores nulos? {hay_nulos}')


Mínimo valor: 2.5737292109196757
Máximo valor: 182.86
¿Hay valores nulos? False


Trabajar con los datos obtenidos directamente de la página web refleja una elección acertada, los datos tienen mayor coherencia. Ahora vamos a guaradar el df en un archivo csv

In [77]:
# Guardar el DataFrame en un archivo CSV
velocidad_media_de_bajada_por_provincia_excel.to_csv('datasets_cleaned/velocidad_media_de_bajada_por_provincia.csv', index=False)

# **Dataset 7:**  
# **Accesos a Internet fijo por velocidad bajada  y provincia**    
Nombre del archivo CSV: accesos_a_internet_fijo_por_velocidad_bajada__y_provincia.csv

In [78]:
pd.set_option('display.max_columns', None)

In [79]:
velocidad_bajada_y_provincia = pd.read_csv('./datasets/accesos_a_internet_fijo_por_velocidad_bajada__y_provincia.csv')

In [80]:
velocidad_bajada_y_provincia.shape

(505, 97)

In [81]:
velocidad_bajada_y_provincia.head()

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps","10,6 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,19 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,11 Mbps","25,5 Mbps",26 Mbps,27 Mbps,28 Mbps,29 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,37 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,47 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps",51 Mbps,"50,25 Mbps",52 Mbps,54 Mbps,55 Mbps,56 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,63 Mbps,64 Mbps,65 Mbps,66 Mbps,67 Mbps,68 Mbps,70 Mbps,71 Mbps,75 Mbps,77 Mbps
0,2022.0,4.0,Buenos Aires,97.634,18,- 0,- 0,28.724,192,4.611,18.736,- 0,- 0,141,21.152,- 0,26,- 0,53.034,- 0,- 0,41.976,14.269,65,32.973,96.230,- 0,10.945,984,64.913,517,204.191,- 0,- 0,10,22.677,- 0,212,44,85.876,258,2,161,- 0,135.315,- 0,3,1,18,88,40.387,- 0,- 0,- 0,- 0,- 0,- 0,- 0,777.236,1.610,16,1,1.365,4,- 0,- 0,- 0,3.790,9,- 0,- 0,- 0,777,1,463.798,- 0,- 0,- 0,- 0,- 0,113,- 0,1,59,435.439,78,1,- 0,- 0,14,- 0,- 0,- 0,5.655,- 0,90.796,- 0
1,2022.0,4.0,Capital Federal,106.772,31,- 0,- 0,451.000,34,1.307,3.789,- 0,- 0,- 0,286,- 0,- 0,- 0,3.564,- 0,- 0,11.975,991,- 0,3.427,11.077,- 0,7,- 0,16.652,3,47.991,- 0,- 0,5,12,- 0,- 0,- 0,29.692,5,- 0,- 0,- 0,6.622,- 0,1,- 0,- 0,- 0,2.370,- 0,- 0,- 0,- 0,- 0,- 0,- 0,58.801,- 0,- 0,- 0,9,- 0,- 0,- 0,30,122,- 0,- 0,- 0,- 0,- 0,- 0,76.732,- 0,- 0,- 0,- 0,- 0,1,- 0,- 0,- 0,105.156,- 0,- 0,- 0,- 0,- 0,1,- 0,- 0,15,- 0,28.406,- 0
2,2022.0,4.0,Catamarca,2.280,- 0,- 0,- 0,34.000,37,- 0,315,- 0,- 0,- 0,115,- 0,- 0,- 0,979.000,- 0,- 0,- 0,- 0,- 0,71,1.551,- 0,1,- 0,- 0,- 0,5.027,- 0,- 0,- 0,407,- 0,- 0,- 0,742,- 0,- 0,- 0,- 0,3.489,- 0,- 0,- 0,- 0,2.228,1,- 0,- 0,- 0,- 0,- 0,- 0,- 0,893,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,30.388,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
3,2022.0,4.0,Chaco,3.909,- 0,225,111,234.000,2,- 0,660,- 0,25,- 0,780,- 0,- 0,- 0,3.909,- 0,- 0,167,385,- 0,3.588,7.213,- 0,106,- 0,702,1,17.037,- 0,- 0,- 0,137,- 0,- 0,- 0,2.808,- 0,- 0,- 0,- 0,3.759,- 0,- 0,- 0,- 0,- 0,747,- 0,- 0,- 0,- 0,- 0,- 0,- 0,2.303,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,18,- 0,- 0,- 0,- 0,- 0,- 0,27.538,- 0,1,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
4,2022.0,4.0,Chubut,20.026,8,- 0,- 0,69.000,32,836,778,- 0,- 0,- 0,5.910,- 0,- 0,- 0,5.397,- 0,- 0,11.441,5.571,- 0,2.407,15.731,- 0,155,- 0,12.646,151,18.055,- 0,- 0,1,14.445,- 0,- 0,32,18.677,- 0,- 0,- 0,- 0,1.573,- 0,- 0,- 0,- 0,2.967,4.134,- 0,- 0,- 0,- 0,- 0,- 0,- 0,9.658,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,18,- 0,- 0,- 0,- 0,140,- 0,17.350,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0


In [82]:
velocidad_bajada_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 97 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Año         504 non-null    float64
 1   Trimestre   504 non-null    float64
 2   Provincia   504 non-null    object 
 3   Otros       504 non-null    object 
 4   0,256 Mbps  504 non-null    object 
 5   0,375 Mbps  504 non-null    object 
 6   0,625 Mbps  504 non-null    object 
 7   0,5 Mbps    504 non-null    float64
 8   0,512 Mbps  504 non-null    object 
 9   0,75 Mbps   504 non-null    object 
 10  1 Mbps      504 non-null    object 
 11  1,2 Mbps    504 non-null    object 
 12  1,25 Mbps   504 non-null    object 
 13  1,5 Mbps    504 non-null    object 
 14  2 Mbps      504 non-null    object 
 15  2,2 Mbps    504 non-null    object 
 16  2,5 Mbps    505 non-null    object 
 17  6,7 Mbps    504 non-null    object 
 18  3 Mbps      504 non-null    float64
 19  3,2 Mbps    504 non-null    o

In [83]:
velocidad_bajada_y_provincia.Provincia.nunique()

24

In [84]:
velocidad_bajada_y_provincia.isnull().sum()

Año           1
Trimestre     1
Provincia     1
Otros         1
0,256 Mbps    1
             ..
68 Mbps       1
70 Mbps       1
71 Mbps       1
75 Mbps       1
77 Mbps       1
Length: 97, dtype: int64

In [85]:
velocidad_bajada_y_provincia[velocidad_bajada_y_provincia.isnull().any(axis=1)]

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps","10,6 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,19 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,11 Mbps","25,5 Mbps",26 Mbps,27 Mbps,28 Mbps,29 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,37 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,47 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps",51 Mbps,"50,25 Mbps",52 Mbps,54 Mbps,55 Mbps,56 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,63 Mbps,64 Mbps,65 Mbps,66 Mbps,67 Mbps,68 Mbps,70 Mbps,71 Mbps,75 Mbps,77 Mbps
504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


No hay datos significativos en esa fila y deseamos eliminarla

In [86]:
velocidad_bajada_y_provincia = velocidad_bajada_y_provincia.dropna()

 Para facilitar la manipulación numérica, se eliminaron las comas en todas las columnas, excepto en la columna 'Provincia', utilizando el método replace.

In [87]:
#Reemplazar comas, puntos y casos específicos por nada en todas las columnas excepto 'Provincia'
velocidad_bajada_y_provincia.iloc[:, 3:] = velocidad_bajada_y_provincia.iloc[:, 3:].replace({',': '', '\.|- 0': ''}, regex=True)

# Convertir todas las columnas excepto 'Provincia' a números flotantes, tratando los valores vacíos como NaN
velocidad_bajada_y_provincia.iloc[:, 3:] = velocidad_bajada_y_provincia.iloc[:, 3:].replace('', np.nan).astype(float)

# Convertir todas las columnas excepto 'Provincia' a enteros, tratando los valores NaN como cero
velocidad_bajada_y_provincia.iloc[:, 3:] = velocidad_bajada_y_provincia.iloc[:, 3:].fillna(0).astype(int)


NameError: name 'np' is not defined

In [88]:
# Convertir las columnas 'Año' y 'Trimestre' a enteros
velocidad_bajada_y_provincia[['Año', 'Trimestre']] = velocidad_bajada_y_provincia[['Año', 'Trimestre']].astype(int)


In [89]:
velocidad_bajada_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 0 to 503
Data columns (total 97 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Año         504 non-null    int32  
 1   Trimestre   504 non-null    int32  
 2   Provincia   504 non-null    object 
 3   Otros       504 non-null    object 
 4   0,256 Mbps  504 non-null    object 
 5   0,375 Mbps  504 non-null    object 
 6   0,625 Mbps  504 non-null    object 
 7   0,5 Mbps    504 non-null    float64
 8   0,512 Mbps  504 non-null    object 
 9   0,75 Mbps   504 non-null    object 
 10  1 Mbps      504 non-null    object 
 11  1,2 Mbps    504 non-null    object 
 12  1,25 Mbps   504 non-null    object 
 13  1,5 Mbps    504 non-null    object 
 14  2 Mbps      504 non-null    object 
 15  2,2 Mbps    504 non-null    object 
 16  2,5 Mbps    504 non-null    object 
 17  6,7 Mbps    504 non-null    object 
 18  3 Mbps      504 non-null    float64
 19  3,2 Mbps    504 non-null    o

En el conjunto de datos constatamos que contiene 505 filas y 97 columnas. Al explorar las primeras filas, notamos la presencia de una fila con datos insignificantes, la cual decidimos eliminar para mantener la integridad del análisis. Implementamos una serie de transformaciones en los valores de las columnas, eliminando comas, puntos y casos específicos, con el objetivo de preparar los datos para un análisis numérico coherente. Luego, convertimos las columnas, excluyendo 'Provincia', a números flotantes, considerando los valores vacíos como NaN. Posteriormente, llevamos a cabo una conversión final a enteros, tratando los valores NaN como cero. 

Después de haber estudiado y transformado las variables que lo necesitaban, procedemos a guardar el DataFrame en un archivo CSV.

In [90]:
# Guardar el DataFrame en un archivo CSV
velocidad_bajada_y_provincia.to_csv('datasets_cleaned/velocidad_bajada_y_provincia.csv', index=False)

# **Dataset 8:**  
# **Acceso a Internet fijo por tecnología y provincia**  
Nombre del archivo CSV: acceso_a_internet_fijo_por_tecnología_y_provincia.csv

In [91]:
acceso_a_internet_fijo_por_tecnologia_y_provincia = pd.read_csv('./datasets/acceso_a_internet_fijo_por_tecnología_y_provincia.csv')

In [92]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.shape

(865, 9)

In [93]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,326.0,3.0,1.0,130.0,72.0,5.0
1,2022,4,Capital Federal,139.0,1.0,139.0,5.0,32.0,2.0
2,2022,4,Catamarca,9.0,11.0,47.0,1.0,2.0,71.0
3,2022,4,Chaco,25.0,61.0,47.0,8.0,3.0,144.0
4,2022,4,Chubut,45.0,74.0,10.0,31.0,9.0,169.0


In [94]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           865 non-null    object 
 1   Trimestre     865 non-null    object 
 2   Provincia     864 non-null    object 
 3   ADSL          864 non-null    float64
 4   Cablemodem    864 non-null    float64
 5   Fibra óptica  864 non-null    float64
 6   Wireless      864 non-null    float64
 7   Otros         864 non-null    float64
 8   Total         864 non-null    float64
dtypes: float64(6), object(3)
memory usage: 60.9+ KB


Observamos los valores nulos

In [95]:
acceso_a_internet_fijo_por_tecnologia_y_provincia[acceso_a_internet_fijo_por_tecnologia_y_provincia.isnull().any(axis=1)]


,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
864,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


La fila no contiene datos con información útil, decidimos eliminar.

In [96]:
# Eliminar la fila con datos provinciales no coincidentes a nivel nacional
acceso_a_internet_fijo_por_tecnologia_y_provincia = acceso_a_internet_fijo_por_tecnologia_y_provincia.drop(864, axis=0)

In [97]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           864 non-null    object 
 1   Trimestre     864 non-null    object 
 2   Provincia     864 non-null    object 
 3   ADSL          864 non-null    float64
 4   Cablemodem    864 non-null    float64
 5   Fibra óptica  864 non-null    float64
 6   Wireless      864 non-null    float64
 7   Otros         864 non-null    float64
 8   Total         864 non-null    float64
dtypes: float64(6), object(3)
memory usage: 60.9+ KB


Convertir los valores numéricos de todas las columnas (excluyendo 'Año', 'Trimestre' y 'Provincia') a formato entero. 

In [98]:
columnas_int = acceso_a_internet_fijo_por_tecnologia_y_provincia.columns.difference(['Año', 'Trimestre', 'Provincia'])

In [99]:
acceso_a_internet_fijo_por_tecnologia_y_provincia[columnas_int] = acceso_a_internet_fijo_por_tecnologia_y_provincia[columnas_int].applymap(lambda x: int(str(x).replace('.', '')))

In [100]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           864 non-null    object
 1   Trimestre     864 non-null    object
 2   Provincia     864 non-null    object
 3   ADSL          864 non-null    int64 
 4   Cablemodem    864 non-null    int64 
 5   Fibra óptica  864 non-null    int64 
 6   Wireless      864 non-null    int64 
 7   Otros         864 non-null    int64 
 8   Total         864 non-null    int64 
dtypes: int64(6), object(3)
memory usage: 60.9+ KB


Revisamos la variable 'Año'

In [101]:
acceso_a_internet_fijo_por_tecnologia_y_provincia['Año'].unique()

array(['2022', '2021', '2020', '2019', '2019 *', '2018', '2017', '2016',
       '2015', '2014'], dtype=object)

Identificamos un error potencial en la columna 'Año', donde algunos valores podrían haber sido afectados por un posible error tipográfico.

In [102]:
# Utilizar apply y una función lambda para reemplazar valores específicos en la columna 'Año'
acceso_a_internet_fijo_por_tecnologia_y_provincia['Año'] = acceso_a_internet_fijo_por_tecnologia_y_provincia['Año'].apply(lambda x: int(x) if x.isdigit() else int(x.split()[0]) if ' *' in x else x)

In [103]:
acceso_a_internet_fijo_por_tecnologia_y_provincia['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

Revisamos la variable 'Trimestre'

In [104]:
acceso_a_internet_fijo_por_tecnologia_y_provincia['Trimestre'].unique()

array(['4', '3', '2', '1', '3 *', '2 *', '1 *'], dtype=object)

Al igual que en'Año' identificamos un error potencial, procedemos a corregirlo

In [105]:
# Utilizar apply y una función lambda para reemplazar valores específicos en la columna 'Trimestre'
acceso_a_internet_fijo_por_tecnologia_y_provincia['Trimestre'] = acceso_a_internet_fijo_por_tecnologia_y_provincia['Trimestre'].apply(lambda x: int(x) if x.isdigit() else int(x.split()[0]) if '*' in x else x)

In [106]:
acceso_a_internet_fijo_por_tecnologia_y_provincia['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

Convertimos las columnas 'Año' y 'Trimestre' a formato entero para facilitar su manipulación y análisis numérico.

In [107]:
acceso_a_internet_fijo_por_tecnologia_y_provincia[['Año', 'Trimestre']] = acceso_a_internet_fijo_por_tecnologia_y_provincia[['Año', 'Trimestre']].astype(int)


In [108]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,3260,30,10,1300,720,50
1,2022,4,Capital Federal,1390,10,1390,50,320,20
2,2022,4,Catamarca,90,110,470,10,20,710
3,2022,4,Chaco,250,610,470,80,30,1440
4,2022,4,Chubut,450,740,100,310,90,1690


In [109]:
acceso_a_internet_fijo_por_tecnologia_y_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           864 non-null    int32 
 1   Trimestre     864 non-null    int32 
 2   Provincia     864 non-null    object
 3   ADSL          864 non-null    int64 
 4   Cablemodem    864 non-null    int64 
 5   Fibra óptica  864 non-null    int64 
 6   Wireless      864 non-null    int64 
 7   Otros         864 non-null    int64 
 8   Total         864 non-null    int64 
dtypes: int32(2), int64(6), object(1)
memory usage: 54.1+ KB


Despues de estudiar y adentrarnos en el contexto de las telecomunicaciones, no encontramos congruencia en los datos vamos a proceder a revisar los datos descargados directamente desde la pagina web.

In [110]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel = pd.read_excel('./datasets_descargados/-10Internet_Accesos-por-tecnologia.xlsx',sheet_name='Accesos Por Tecnología')

In [111]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,325908.0,2767247.0,1499149.0,129910.0,72451.0,4794665.0
1,2022,4,Capital Federal,138753.0,1238196.0,138548.0,4603.0,31826.0,1551926.0
2,2022,4,Catamarca,8952.0,10998.0,47337.0,1379.0,2243.0,70909.0
3,2022,4,Chaco,24582.0,61265.0,47285.0,7957.0,2574.0,143663.0
4,2022,4,Chubut,45450.0,73923.0,9815.0,30721.0,8850.0,168759.0
...,...,...,...,...,...,...,...,...,...
861,2014,1,Santiago Del Estero,32567.0,3598.0,19.0,915.0,34.0,37133.0
862,2014,1,Tierra Del Fuego,21618.0,2837.0,648.0,1.0,2934.0,28038.0
863,2014,1,Tucumán,129717.0,83.0,121.0,13.0,98.0,130032.0
864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           865 non-null    object 
 1   Trimestre     865 non-null    object 
 2   Provincia     864 non-null    object 
 3   ADSL          864 non-null    float64
 4   Cablemodem    864 non-null    float64
 5   Fibra óptica  864 non-null    float64
 6   Wireless      864 non-null    float64
 7   Otros         864 non-null    float64
 8   Total         864 non-null    float64
dtypes: float64(6), object(3)
memory usage: 61.0+ KB


Observamos los valores nulos.

In [113]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel[acceso_a_internet_fijo_por_tecnologia_y_provincia_excel.isnull().any(axis=1)]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
865,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
# Eliminar filas con valores nulos en el DataFrame
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel = acceso_a_internet_fijo_por_tecnologia_y_provincia_excel.dropna()

Revisamos nuevamente las variables:

In [115]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Año'].unique()

array([2022, 2021, 2020, 2019, '2019 *', 2018, 2017, 2016, 2015, 2014],
      dtype=object)

In [116]:
# Utilizar apply y una función lambda para reemplazar valores específicos en la columna 'Año'
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Año'] = acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Año'].apply(
    lambda x: int(x) if (isinstance(x, str) and x.isdigit()) else int(x.split()[0]) if (isinstance(x, str) and ' *' in x) else x
)

In [117]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [118]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 864 entries, 0 to 863
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           864 non-null    int64  
 1   Trimestre     864 non-null    object 
 2   Provincia     864 non-null    object 
 3   ADSL          864 non-null    float64
 4   Cablemodem    864 non-null    float64
 5   Fibra óptica  864 non-null    float64
 6   Wireless      864 non-null    float64
 7   Otros         864 non-null    float64
 8   Total         864 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 67.5+ KB


Aplicamos lo mismo para la Variable Trimestre, como hicimos anteriormente:

In [119]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Trimestre'].unique()

array([4, 3, 2, 1, '3 *', '2 *', '1 *'], dtype=object)

In [120]:
# Utilizar apply y una función lambda para reemplazar valores específicos en la columna 'Año'
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Trimestre'] = acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Trimestre'].apply(
    lambda x: int(x) if (isinstance(x, str) and x.isdigit()) else int(x.split()[0]) if (isinstance(x, str) and ' *' in x) else x
)

In [121]:
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

Ahora procedemos a guardar en csv la data limpia y procesada:

In [122]:
# Guardar el DataFrame en un archivo CSV
acceso_a_internet_fijo_por_tecnologia_y_provincia_excel.to_csv('datasets_cleaned/acceso_a_internet_fijo_por_tecnologia_y_provincia.csv', index=False)

# **Dataset 9:**  
# **Total nacional de accesos a Internet fijo por tipo de tecnología**    
Nombre del archivo CSV: total_nacional_de_accesos_a_internet_fijo_por_tipo_de_tecnología.csv

In [123]:
total_nacional_por_tipo_de_tecnologia = pd.read_csv('./datasets/total_nacional_de_accesos_a_internet_fijo_por_tipo_de_tecnología.csv')

In [124]:
total_nacional_por_tipo_de_tecnologia.shape

(36, 9)

In [125]:
total_nacional_por_tipo_de_tecnologia.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,4,1,6,3,573,222,11,Oct-Dic 2022
1,2022,3,1,6,3,557,235,11,Jul-Sept 2022
2,2022,2,1,6,3,556,232,11,Abr-Jun 2022
3,2022,1,2,6,2,546,252,11,Ene-Mar 2022
4,2021,4,2,6,2,523,253,10,Oct-Dic 2021


In [126]:
total_nacional_por_tipo_de_tecnologia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           36 non-null     int64 
 1   Trimestre     36 non-null     int64 
 2   ADSL          36 non-null     int64 
 3   Cablemodem    36 non-null     int64 
 4   Fibra óptica  36 non-null     int64 
 5   Wireless      36 non-null     int64 
 6   Otros         36 non-null     int64 
 7   Total         36 non-null     int64 
 8   Periodo       36 non-null     object
dtypes: int64(8), object(1)
memory usage: 2.7+ KB


Después de analizar detenidamente los datos y comprender el contexto subyacente, identificamos discrepancias en los valores de las variables. Aquellos obtenidos directamente de la página web en formato Excel resultaron ser más coherentes. Ante esta observación, hemos decidido enfocarnos en el uso exclusivo de los datos extraídos directamente de la fuente web, procesándolos con especial atención. Esta elección busca garantizar una representación más precisa y coherente en nuestro conjunto de datos, priorizando la fiabilidad de la información proveniente de la descarga directa.

In [127]:
total_nacional_por_tipo_de_tecnologia_excel = pd.read_excel('./datasets_descargados/-10Internet_Accesos-por-tecnologia.xlsx')

In [128]:
total_nacional_por_tipo_de_tecnologia_excel.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,4,1315373,6037457,3060575,572969,221740,11208114,Oct-Dic 2022
1,2022,3,1395277,6031970,2871541,557110,235230,11091128,Jul-Sept 2022
2,2022,2,1468333,5979214,2723285,556243,231609,10958684,Abr-Jun 2022
3,2022,1,1533240,6073426,2219533,545814,251996,10624009,Ene-Mar 2022
4,2021,4,1657615,5984240,2072236,523107,252596,10489794,Oct-Dic 2021


In [129]:
total_nacional_por_tipo_de_tecnologia_excel.isnull().sum()

Año             0
Trimestre       0
ADSL            0
Cablemodem      0
Fibra óptica    0
Wireless        0
Otros           0
Total           0
Periodo         0
dtype: int64

Después de revisar y obtener una primera impresión de los datos, llegamos a la conclusión de que la decisión de cambiar la información fue acertada. Durante el primer chequeo, notamos que la data no estaba configurada de manera óptima para llevar a cabo los análisis necesarios. Por lo tanto, decidimos extraer los datos directamente desde la página web como una alternativa. Esta nueva fuente de datos proveniente de la página web se presenta en condiciones óptimas y no requiere ninguna modificación adicional, consolidando así la validez de nuestra elección.

In [130]:
# Guardar el DataFrame en un archivo CSV en la carpeta 'datasets_cleaned'
total_nacional_por_tipo_de_tecnologia_excel.to_csv('datasets_cleaned/total_nacional_por_tipo_de_tecnologia_excel.csv', index=False)

# **Dataset 10:**   
# **Ingresos por la operación del servicio de Internet fijo**  
Nombre del archivo CSV: ingresos_por_la_operación_del_servicio_de_internet_fijo.csv

In [131]:
 ingresos_por_operacion_del_servicio = pd.read_csv('./datasets/ingresos_por_la_operación_del_servicio_de_internet_fijo.csv')

In [132]:
 ingresos_por_operacion_del_servicio.shape

(36, 4)

In [133]:
 ingresos_por_operacion_del_servicio

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,4,72,Oct-Dic 2022
1,2022,3,64,Jul-Sept 2022
2,2022,2,60,Abr-Jun 2022
3,2022,1,55,Ene-Mar 2022
4,2021,4,45,Oct-Dic 2021
5,2021,3,43,Jul-Sept 2021
6,2021,2,38,Abr-Jun 2021
7,2021,1,37,Ene-Mar 2021
8,2020,4,34,Oct-Dic 2020
9,2020,3,32,Jul-Sept 2020


In [134]:
 ingresos_por_operacion_del_servicio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Año                        36 non-null     int64 
 1   Trimestre                  36 non-null     int64 
 2   Ingresos (miles de pesos)  36 non-null     int64 
 3   Periodo                    36 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.3+ KB


In [135]:
 ingresos_por_operacion_del_servicio.groupby('Año')['Ingresos (miles de pesos)'].sum().reset_index()

,Año,Ingresos (miles de pesos)
0,2014,13
1,2015,20
2,2016,27
3,2017,40
4,2018,59
5,2019,87
6,2020,128
7,2021,163
8,2022,251


Al realizar descargas directas desde la API y comparar los datos con aquellos obtenidos mediante descargas directas desde la página en formato Excel, hemos observado que en la mayoría de los casos, los conjuntos de datos son idénticos o muestran similitudes notables. Sin embargo, en este escenario específico, hemos identificado diferencias significativas. Ante esta disparidad, hemos optado por utilizar los datos descargados directamente desde la página web para asegurar la precisión y coherencia de la información que será empleada en nuestros análisis y conclusiones.

In [136]:
ingresos_por_operacion_del_servicio_excel = pd.read_excel('./datasets_descargados/Internet_Ingresos.xlsx')

In [137]:
ingresos_por_operacion_del_servicio_excel.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,4,7.244348e+07,Oct-Dic 2022
1,2022,3,6.408678e+07,Jul-Sept 2022
2,2022,2,6.041950e+07,Abr-Jun 2022
3,2022,1,5.521931e+07,Ene-Mar 2022
4,2021,4,4.546789e+07,Oct-Dic 2021


In [138]:
ingresos_por_operacion_del_servicio_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        36 non-null     int64  
 1   Trimestre                  36 non-null     int64  
 2   Ingresos (miles de pesos)  36 non-null     float64
 3   Periodo                    36 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ KB


In [139]:
ingresos_por_operacion_del_servicio_excel['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [140]:
ingresos_por_operacion_del_servicio_excel['Trimestre'].unique()

array([4, 3, 2, 1], dtype=int64)

La variable "Periodo" no proporciona información importante. La eliminaremos del conjunto de datos

In [141]:
# Eliminar la columna 'Periodo'
ingresos_por_operacion_del_servicio_excel.drop('Periodo', axis=1, inplace=True)

Reemplazamos los puntos (.) por nada y luego convertimos la columna "Ingresos (miles de pesos)" a enteros.

In [142]:
# Reemplazar los puntos por nada en la columna 'Ingresos (miles de pesos)'
ingresos_por_operacion_del_servicio_excel['Ingresos (miles de pesos)'] = ingresos_por_operacion_del_servicio_excel['Ingresos (miles de pesos)'].replace('\.', '', regex=True)

# Convertir la columna 'Ingresos (miles de pesos)' a enteros
ingresos_por_operacion_del_servicio_excel['Ingresos (miles de pesos)'] = ingresos_por_operacion_del_servicio_excel['Ingresos (miles de pesos)'].astype(int)


In [143]:
ingresos_por_operacion_del_servicio_excel.head()

,Año,Trimestre,Ingresos (miles de pesos)
0,2022,4,72443482
1,2022,3,64086784
2,2022,2,60419500
3,2022,1,55219313
4,2021,4,45467887


Luego de realizar un análisis detallado y efectuar ajustes necesarios tras una revisión inicial de los datos, nos disponemos a cargar la versión modificada.

In [144]:
# Guardar el DataFrame modificado en un archivo CSV
ingresos_por_operacion_del_servicio_excel.to_csv('datasets_cleaned/ingresos_por_operacion_del_servicio.csv', index=False)


# **Dataset 11:**  
# Accesos a Internet fijo por tecnología y localidad  
Nombre del archivo CSV: accesos_a_internet_fijo_por_tecnología_y_localidad.csv

In [145]:
accesos_por_tecnología_y_localidad = pd.read_csv('./datasets/accesos_a_internet_fijo_por_tecnología_y_localidad.csv')

In [146]:
accesos_por_tecnología_y_localidad.shape

(3137, 14)

In [147]:
accesos_por_tecnología_y_localidad.head()

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,Unnamed: 13
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,874,"4,54",- 0,3,- 0,852,- 0,706,"6,98",NaN
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,- 0,- 0,- 0,10,1,- 0,- 0,192,NaN
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,- 0,- 0,- 0,- 0,- 0,- 0,- 0,181,181,NaN
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,- 0,782,- 0,6,167,- 0,- 0,327,"1,28",NaN
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,- 0,- 0,- 0,- 0,- 0,- 0,- 0,6,6,NaN


In [148]:
accesos_por_tecnología_y_localidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137 entries, 0 to 3136
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Provincia      3137 non-null   object 
 1   Partido        3137 non-null   object 
 2   Localidad      3137 non-null   object 
 3   Link Indec     3137 non-null   object 
 4   ADSL           3137 non-null   object 
 5   CABLEMODEM     3137 non-null   object 
 6   DIAL UP        3137 non-null   object 
 7   FIBRA OPTICA   3137 non-null   object 
 8   OTROS          3137 non-null   object 
 9   SATELITAL      3137 non-null   object 
 10  WIMAX          3137 non-null   object 
 11  WIRELESS       3137 non-null   object 
 12  Total general  3137 non-null   object 
 13  Unnamed: 13    0 non-null      float64
dtypes: float64(1), object(13)
memory usage: 343.2+ KB


Como en el Datasaet anterior, en este caso tambien  hemos identificado diferencias significativas. Ante esta disparidad, hemos optado por utilizar los datos descargados directamente desde la página web para asegurar la precisión y coherencia de la información que será empleada en nuestros análisis y conclusiones.

In [149]:
# Cargar el archivo Excel en un DataFrame
accesos_por_tecnología_y_localidad_excel = pd.read_excel('./datasets_descargados/-12Internet-Accesos-por-tecno_por_loc.xlsx')

In [150]:
accesos_por_tecnología_y_localidad_excel.head()

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,874,4542,0,3,0,852,0,706,6977
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,0,0,0,10,1,0,0,192
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,0,0,0,0,0,0,0,181,181
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,0,782,0,6,167,0,0,327,1282
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,0,0,0,0,0,0,0,6,6


Revisamos la variable `Link Indec` y observamos el numero total de valores faltantes y duplicados 

In [151]:
accesos_por_tecnología_y_localidad_excel[accesos_por_tecnología_y_localidad_excel['Link Indec'] == 'Sin Datos'].shape[0]

72

In [152]:
accesos_por_tecnología_y_localidad_excel['Link Indec'].duplicated().sum()

423

llevó a cabo la sustitución de los valores '- 0' por el valor correspondiente '0' donde era aplicable. Además, se realizaron conversiones de tipo de datos según fuera necesario. Es importante señalar que el campo 'Link Indec' contiene 424 registros con el valor "Sin Datos", por lo tanto, se mantiene este campo como tipo 'object' en lugar de convertirlo a 'int', como sería apropiado.

Esta información es crucial para futuros análisis, y cabe destacar que hay 72 registros en los cuales el campo 'Link Indec' tiene el valor 'Sin Datos'. Asimismo, se identificaron 423 duplicados en el campo 'Link Indec' dentro del conjunto de datos. Estos detalles proporcionan un contexto valioso para tener en cuenta en futuras investigaciones y procesamientos de datos.

Luego de efectuar estos ajustes necesarios en los datos, nos disponemos a cargar la versión modificada.

In [222]:
# Guardar el DataFrame modificado en un archivo CSV
accesos_por_tecnología_y_localidad_excel.to_csv('datasets_cleaned/accesos_por_tecnología_y_localidad.csv', index=False)


# **Dataset 12:**  
# **Accesos a Internet fijo por velocidad de bajada y localidad**  
Nombre del archivo CSV: accesos_a_internet_fijo_por_velocidad_de_bajada_y_localidad.csv 

In [154]:
accesos_por_velocidad_de_bajada_y_localidad = pd.read_csv('./datasets/accesos_a_internet_fijo_por_velocidad_de_bajada_y_localidad.csv')

In [155]:
accesos_por_velocidad_de_bajada_y_localidad.shape

(3095, 83)

In [156]:
accesos_por_velocidad_de_bajada_y_localidad.head()

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,375 Mbps","0,5 Mbps","0,512 Mbps","0,625 Mbps","0,75 Mbps",1 Mbps,"1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,5 Mbps",26 Mbps,27 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps","50,25 Mbps",51 Mbps,55 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,64 Mbps,65 Mbps,66 Mbps,70 Mbps,75 Mbps,78 Mbps,80 Mbps,83 Mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,NaN,NaN,NaN,2.0,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,145.0,NaN,NaN,NaN,194.0,NaN,NaN,NaN,299.0,NaN,177.0,NaN,NaN,3.965,NaN,NaN,NaN,415.0,NaN,NaN,NaN,595.0,NaN,NaN,NaN,NaN,NaN,166.0,NaN,NaN,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,NaN,3.0,NaN,NaN,56.0,NaN,7.0,3.0,NaN,NaN,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
accesos_por_velocidad_de_bajada_y_localidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 83 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Provincia   3095 non-null   object 
 1   Partido     3095 non-null   object 
 2   Localidad   3095 non-null   object 
 3   Link Indec  3095 non-null   object 
 4   Otros       962 non-null    float64
 5   0,256 Mbps  28 non-null     float64
 6   0,375 Mbps  1 non-null      float64
 7   0,5 Mbps    643 non-null    float64
 8   0,512 Mbps  523 non-null    float64
 9   0,625 Mbps  1 non-null      float64
 10  0,75 Mbps   284 non-null    float64
 11  1 Mbps      891 non-null    float64
 12  1,25 Mbps   1 non-null      float64
 13  1,5 Mbps    15 non-null     float64
 14  2 Mbps      722 non-null    float64
 15  2,2 Mbps    1 non-null      float64
 16  2,5 Mbps    4 non-null      float64
 17  3 Mbps      1351 non-null   float64
 18  3,3 Mbps    1 non-null      float64
 19  3,5 Mbps    373 non-null   

Nuevamente procedemos a tomar los datos del dataset descagado desde la pagina directamente

In [158]:
accesos_por_velocidad_de_bajada_y_localidad_excel = pd.read_excel('./datasets_descargados/-13Internet_Accesos-por-velocidad_por loc.xlsx', thousands=',')

In [159]:
accesos_por_velocidad_de_bajada_y_localidad_excel

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,375 Mbps","0,5 Mbps","0,512 Mbps","0,625 Mbps","0,75 Mbps",1 Mbps,"1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,5 Mbps",26 Mbps,27 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps","50,25 Mbps",51 Mbps,55 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,64 Mbps,65 Mbps,66 Mbps,70 Mbps,75 Mbps,78 Mbps,80 Mbps,83 Mbps,85 Mbps,90 Mbps,92 Mbps,95 Mbps,100 Mbps,"100,25 Mbps",110 Mbps,120 Mbps,122 Mbps,125 Mbps,126 Mbps,130 Mbps,140 Mbps,150 Mbps,157 Mbps,158 Mbps,160 Mbps,164 Mbps,165 Mbps,180 Mbps,200 Mbps,212 Mbps,220 Mbps,225 Mbps,234 Mbps,240 Mbps,250 Mbps,256 Mbps,275 Mbps,300 Mbps,320 Mbps,325 Mbps,330 Mbps,340 Mbps,400 Mbps,450 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,950 Mbps,1000 Mbps,1024 Mbps,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,NaN,NaN,NaN,2.0,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,145.0,NaN,NaN,NaN,194.0,NaN,NaN,NaN,299.0,NaN,177.0,NaN,NaN,3965.0,NaN,NaN,NaN,415.0,NaN,NaN,NaN,595.0,NaN,NaN,NaN,NaN,NaN,166.0,NaN,NaN,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6208.0
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,NaN,3.0,NaN,NaN,56.0,NaN,7.0,3.0,NaN,NaN,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [160]:
accesos_por_velocidad_de_bajada_y_localidad_excel.shape

(3095, 134)

Identificamos variables que contenían valores nulos. Con el objetivo de asegurar la integridad de nuestros análisis y facilitar su manejo, procedimos a reemplazar todos los valores nulos por cero (0). llevamos a cabo la conversión de estas variables a formato entero.

In [161]:
columnas_int = accesos_por_velocidad_de_bajada_y_localidad_excel.columns.difference(['Provincia', 'Partido', 'Localidad', 'Link Indec'])
accesos_por_velocidad_de_bajada_y_localidad_excel[columnas_int] = accesos_por_velocidad_de_bajada_y_localidad_excel[columnas_int].fillna(0)

In [162]:
accesos_por_velocidad_de_bajada_y_localidad_excel[columnas_int] = accesos_por_velocidad_de_bajada_y_localidad_excel[columnas_int].astype(int)

In [163]:
accesos_por_velocidad_de_bajada_y_localidad_excel.head()

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,375 Mbps","0,5 Mbps","0,512 Mbps","0,625 Mbps","0,75 Mbps",1 Mbps,"1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,5 Mbps",26 Mbps,27 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps","50,25 Mbps",51 Mbps,55 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,64 Mbps,65 Mbps,66 Mbps,70 Mbps,75 Mbps,78 Mbps,80 Mbps,83 Mbps,85 Mbps,90 Mbps,92 Mbps,95 Mbps,100 Mbps,"100,25 Mbps",110 Mbps,120 Mbps,122 Mbps,125 Mbps,126 Mbps,130 Mbps,140 Mbps,150 Mbps,157 Mbps,158 Mbps,160 Mbps,164 Mbps,165 Mbps,180 Mbps,200 Mbps,212 Mbps,220 Mbps,225 Mbps,234 Mbps,240 Mbps,250 Mbps,256 Mbps,275 Mbps,300 Mbps,320 Mbps,325 Mbps,330 Mbps,340 Mbps,400 Mbps,450 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,950 Mbps,1000 Mbps,1024 Mbps,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0,0,0,2,0,0,19,0,0,0,0,0,0,85,0,145,0,0,0,194,0,0,0,299,0,177,0,0,3965,0,0,0,415,0,0,0,595,0,0,0,0,0,166,0,0,0,0,96,0,0,0,0,0,0,0,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6208
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,0,0,0,1,0,0,0,0,0,0,0,0,0,181,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,0,0,0,0,0,0,0,0,0,0,0,0,0,66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,0,0,0,0,0,0,0,0,0,0,106,0,0,3,0,0,56,0,7,3,0,0,0,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,181


Revisamos los valores duplicados

In [164]:
accesos_por_velocidad_de_bajada_y_localidad_excel[accesos_por_velocidad_de_bajada_y_localidad_excel.duplicated()]

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,375 Mbps","0,5 Mbps","0,512 Mbps","0,625 Mbps","0,75 Mbps",1 Mbps,"1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,20 Mbps,"20,5 Mbps",21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,5 Mbps",26 Mbps,27 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,48 Mbps,49 Mbps,50 Mbps,"50,1 Mbps","50,25 Mbps",51 Mbps,55 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,64 Mbps,65 Mbps,66 Mbps,70 Mbps,75 Mbps,78 Mbps,80 Mbps,83 Mbps,85 Mbps,90 Mbps,92 Mbps,95 Mbps,100 Mbps,"100,25 Mbps",110 Mbps,120 Mbps,122 Mbps,125 Mbps,126 Mbps,130 Mbps,140 Mbps,150 Mbps,157 Mbps,158 Mbps,160 Mbps,164 Mbps,165 Mbps,180 Mbps,200 Mbps,212 Mbps,220 Mbps,225 Mbps,234 Mbps,240 Mbps,250 Mbps,256 Mbps,275 Mbps,300 Mbps,320 Mbps,325 Mbps,330 Mbps,340 Mbps,400 Mbps,450 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,950 Mbps,1000 Mbps,1024 Mbps,Total general


No se observan Valores Duplicados.

Después de echar un primer vistazo a los datos y realizar algunos ajustes necesarios, estamos listos para guardar el archivo en formato CSV.

In [165]:
# Guardar el DataFrame modificado en un archivo CSV
accesos_por_velocidad_de_bajada_y_localidad_excel.to_csv('datasets_cleaned/accesos_por_velocidad_de_bajada_y_localidad.csv', index=False)


# **Dataset 13:**  
# **Velocidad media de bajada de Internet (nacional)**  
Nombre del archivo CSV: velocidad_media_de_bajada_de_internet_(nacional).csv

In [166]:
velocidad_media_de_bajada_nacional = pd.read_csv('./datasets/velocidad_media_de_bajada_de_internet_(nacional).csv')

In [167]:
velocidad_media_de_bajada_nacional.shape

(36, 4)

In [168]:
velocidad_media_de_bajada_nacional

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2022,4,10.656,Oct-Dic 2022
1,2022,3,6.246,Jul-Sept 2022
2,2022,2,5.844,Abr-Jun 2022
3,2022,1,5.511,Ene-Mar 2022
4,2021,4,5.234,Oct-Dic 2021
5,2021,3,4.846,Jul-Sept 2021
6,2021,2,4.563,Abr-Jun 2021
7,2021,1,4.311,Ene-Mar 2021
8,2020,4,4.236,Oct-Dic 2020
9,2020,3,4.067,Jul-Sept 2020


In [169]:
velocidad_media_de_bajada_nacional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     36 non-null     int64  
 1   Trimestre               36 non-null     int64  
 2   Mbps (Media de bajada)  36 non-null     float64
 3   Trimestre.1             36 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ KB


Como en el Datasaet anterior, en este caso tambien hemos identificado diferencias significativas ne la variable Mbps (Media de bajada). Ante esta disparidad, hemos optado por utilizar los datos descargados directamente desde la página web para asegurar la precisión y coherencia de la información que será empleada en nuestros análisis y conclusiones.

In [170]:
velocidad_media_de_bajada_nacional_excel = pd.read_excel('./datasets_descargados/-14historico_velocidad_internet.xlsx', sheet_name='Totales')

In [171]:
velocidad_media_de_bajada_nacional_excel

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2022,4,106.560000,Oct-Dic 2022
1,2022,3,62.460000,Jul-Sept 2022
2,2022,2,58.440000,Abr-Jun 2022
3,2022,1,55.110000,Ene-Mar 2022
4,2021,4,52.340000,Oct-Dic 2021
5,2021,3,48.460000,Jul-Sept 2021
6,2021,2,45.630000,Abr-Jun 2021
7,2021,1,43.110000,Ene-Mar 2021
8,2020,4,42.360000,Oct-Dic 2020
9,2020,3,40.670000,Jul-Sept 2020


In [172]:
velocidad_media_de_bajada_nacional_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     36 non-null     int64  
 1   Trimestre               36 non-null     int64  
 2   Mbps (Media de bajada)  36 non-null     float64
 3   Trimestre.1             36 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ KB


Despues de cambiar el dataset, el dataset descargado de la pagina web directamente en un primer vistazo a los datos y  no fue necesario realizar ajustes necesarios, procedemos a guardar el archivo en formato CSV.

In [173]:
# Guardar el DataFrame modificado en un archivo CSV
velocidad_media_de_bajada_nacional_excel.to_csv('datasets_cleaned/velocidad_media_de_bajada_nacional.csv', index=False)


# **Dataset 14:**  
# **Accesos a banda ancha y banda angosta por provincia**  
Nombre del archivo CSV: accesos_a_banda_ancha_y_banda_angosta_por_provincia.csv

In [174]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia = pd.read_csv('./datasets/accesos_a_banda_ancha_y_banda_angosta_por_provincia.csv', decimal=',', thousands='.')

In [175]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia.shape

(864, 6)

In [176]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,4,Buenos Aires,5,60.0,5
1,2022,4,Capital Federal,2,20.0,2
2,2022,4,Catamarca,71,10.0,71
3,2022,4,Chaco,144,50.0,144
4,2022,4,Chubut,168,9040.0,169
...,...,...,...,...,...,...
859,2014,1,Santa Cruz,26,4600.0,27
860,2014,1,Santa Fe,506,6120.0,507
861,2014,1,Santiago Del Estero,37,90.0,37
862,2014,1,Tierra Del Fuego,27,7660.0,28


In [177]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               864 non-null    int64  
 1   Trimestre         864 non-null    int64  
 2   Provincia         864 non-null    object 
 3   Banda ancha fija  864 non-null    int64  
 4   Dial up           862 non-null    float64
 5   Total             864 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 40.6+ KB


Filtrar filas con valores nulos en la columna `Dial up`

In [178]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia[accesos_a_banda_ancha_y_banda_angosta_por_provincia['Dial up'].isna()]

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
440,2018,2,Formosa,45,NaN,45
445,2018,2,Misiones,106,NaN,106


Llenar valores nulos en la columna `Dial up` con cero

In [179]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia['Dial up'].fillna(0, inplace=True)

In [180]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               864 non-null    int64  
 1   Trimestre         864 non-null    int64  
 2   Provincia         864 non-null    object 
 3   Banda ancha fija  864 non-null    int64  
 4   Dial up           864 non-null    float64
 5   Total             864 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 40.6+ KB


In [181]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia['Año'].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], dtype=int64)

In [182]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia['Provincia'].unique()

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Córdoba', 'Corrientes', 'Entre Ríos', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén',
       'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago Del Estero', 'Tierra Del Fuego', 'Tucumán'],
      dtype=object)

In [183]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia.Provincia.nunique()

24

In [184]:
valores_unicos_ordenados = valores_unicos=accesos_a_banda_ancha_y_banda_angosta_por_provincia['Dial up'].unique()
valores_unicos_ordenados.sort()

print("Valores únicos de 'Dial up' ordenados de menor a mayor:")
print(valores_unicos_ordenados)

Valores únicos de 'Dial up' ordenados de menor a mayor:
[   0.   10.   20.   30.   40.   50.   60.   70.   80.   90.  100.  110.
  120.  130.  140.  150.  160.  170.  180.  200.  210.  220.  230.  240.
  250.  260.  280.  300.  310.  320.  330.  340.  360.  370.  410.  420.
  430.  440.  450.  480.  490.  500.  510.  540.  550.  570.  620.  640.
  650.  680.  740.  770.  870.  880.  920.  940.  950. 1010. 1050. 1070.
 1080. 1100. 1150. 1160. 1190. 1330. 1370. 1480. 1500. 1510. 1710. 1720.
 1730. 1760. 1790. 1800. 1820. 1830. 1850. 1870. 1920. 1970. 1980. 2000.
 2040. 2050. 2100. 2170. 2260. 2270. 2280. 2290. 2310. 2340. 2350. 2360.
 2370. 2440. 2460. 2560. 2590. 2680. 2690. 2810. 2890. 2900. 2910. 2940.
 3000. 3090. 3110. 3120. 3130. 3150. 3170. 3190. 3230. 3250. 3280. 3540.
 3560. 3590. 3620. 3630. 3640. 3690. 3700. 3710. 3730. 3760. 3790. 3860.
 3960. 4100. 4240. 4340. 4350. 4360. 4460. 4490. 4550. 4590. 4600. 4610.
 4680. 4710. 4770. 4850. 4870. 4950. 5000. 5010. 5040. 5080. 5420. 5

Hemos identificado inconsistencia en los datos. vamos a  optar por utilizar los datos descargados directamente desde la página web para asegurar la precisión y coherencia de la información que será empleada en nuestros análisis y conclusiones.

In [185]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel = pd.read_excel('./datasets_descargados/-15Internet_BAF.xlsx')

In [239]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,4,Buenos Aires,4788466,6199.0,4794665
1,2022,4,Capital Federal,1549781,2145.0,1551926
2,2022,4,Catamarca,70908,1.0,70909
3,2022,4,Chaco,143658,5.0,143663
4,2022,4,Chubut,167855,904.0,168759
...,...,...,...,...,...,...
859,2014,1,Santa Cruz,26304,460.0,26764
860,2014,1,Santa Fe,506000,612.0,506612
861,2014,1,Santiago Del Estero,37124,9.0,37133
862,2014,1,Tierra Del Fuego,27272,766.0,28038


In [187]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               864 non-null    int64  
 1   Trimestre         864 non-null    int64  
 2   Provincia         864 non-null    object 
 3   Banda ancha fija  864 non-null    int64  
 4   Dial up           862 non-null    float64
 5   Total             864 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 40.6+ KB


Observamos un poco los datos los valores minimo, maximos y si hay algunos valores nulos 

In [188]:
# Obtener información sobre la variable "Banda ancha fija"
info_banda_ancha = accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel['Banda ancha fija']
minimo_banda_ancha = info_banda_ancha.min()
maximo_banda_ancha = info_banda_ancha.max()
valores_nulos_banda_ancha = info_banda_ancha.isnull().sum()

# Obtener información sobre la variable "Dial up"
info_dial_up = accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel['Dial up']
minimo_dial_up = info_dial_up.min()
maximo_dial_up = info_dial_up.max()
valores_nulos_dial_up = info_dial_up.isnull().sum()

# Imprimir los resultados
print("Banda ancha fija:")
print(f"Valor mínimo: {minimo_banda_ancha}")
print(f"Valor máximo: {maximo_banda_ancha}")
print(f"¿Tiene valores nulos? {valores_nulos_banda_ancha > 0}")
print("\nDial up:")
print(f"Valor mínimo: {minimo_dial_up}")
print(f"Valor máximo: {maximo_dial_up}")
print(f"¿Tiene valores nulos? {valores_nulos_dial_up > 0}")


Banda ancha fija:
Valor mínimo: 12193
Valor máximo: 4788466
¿Tiene valores nulos? False

Dial up:
Valor mínimo: 0.0
Valor máximo: 15229.0
¿Tiene valores nulos? True


In [189]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel['Dial up'].isnull().sum()

2

In [190]:
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel.loc[
    accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel['Dial up'].isnull()
]

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
440,2018,2,Formosa,44554,NaN,44554
445,2018,2,Misiones,106147,NaN,106147


Después de echar un primer vistazo a los datos y realizar algunos ajustes necesarios, estamos listos para guardar el archivo en formato CSV.

In [191]:
# Guardar el DataFrame modificado en un archivo CSV
accesos_a_banda_ancha_y_banda_angosta_por_provincia_excel.to_csv('datasets_cleaned/accesos_a_banda_ancha_y_banda_angosta_por_provincia.csv', index=False)

# **Dataset 15:**   
# Total nacional de accesos a Internet fijo por banda ancha y banda angosta  
Nombre del archivo CSV: total_nacional_de_accesos_a_internet_fijo_por_banda_ancha_y_banda_angosta.csv

In [192]:
nacional_por_banda_ancha_y_banda_angosta = pd.read_csv('./datasets/total_nacional_de_accesos_a_internet_fijo_por_banda_ancha_y_banda_angosta.csv')

In [193]:
nacional_por_banda_ancha_y_banda_angosta.shape

(36, 6)

In [194]:
nacional_por_banda_ancha_y_banda_angosta.head()

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,4,11,12,11,Oct-Dic 2022
1,2022,3,11,12,11,Jul-Sept 2022
2,2022,2,11,12,11,Abr-Jun 2022
3,2022,1,11,13,11,Ene-Mar 2022
4,2021,4,10,13,10,Oct-Dic 2021


In [195]:
nacional_por_banda_ancha_y_banda_angosta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               36 non-null     int64 
 1   Trimestre         36 non-null     int64 
 2   Banda ancha fija  36 non-null     int64 
 3   Dial up           36 non-null     int64 
 4   Total             36 non-null     int64 
 5   Periodo           36 non-null     object
dtypes: int64(5), object(1)
memory usage: 1.8+ KB


Hemos identificado diferencias significativas. Ante esta disparidad, hemos optado por utilizar los datos descargados directamente desde la página web para asegurar la precisión y coherencia de la información que será empleada en nuestros análisis y conclusiones.

In [196]:
nacional_por_banda_ancha_y_banda_angosta_excel = pd.read_excel('./datasets_descargados/-15Internet_BAF.xlsx', sheet_name='Totales')

In [197]:
nacional_por_banda_ancha_y_banda_angosta_excel.head()

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,4,11195668,12446,11208114,Oct-Dic 2022
1,2022,3,11078691,12437,11091128,Jul-Sept 2022
2,2022,2,10946248,12436,10958684,Abr-Jun 2022
3,2022,1,10611390,12619,10624009,Ene-Mar 2022
4,2021,4,10476933,12861,10489794,Oct-Dic 2021


In [198]:
nacional_por_banda_ancha_y_banda_angosta_excel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               36 non-null     int64 
 1   Trimestre         36 non-null     int64 
 2   Banda ancha fija  36 non-null     int64 
 3   Dial up           36 non-null     int64 
 4   Total             36 non-null     int64 
 5   Periodo           36 non-null     object
dtypes: int64(5), object(1)
memory usage: 1.8+ KB


Después de echar un primer vistazo a los datos y realizar algunos ajustes necesarios, estamos listos para guardar el archivo en formato CSV.

In [199]:
# Guardar el DataFrame modificado en un archivo CSV
nacional_por_banda_ancha_y_banda_angosta_excel.to_csv('datasets_cleaned/nacional_por_banda_ancha_y_banda_angosta.csv', index=False)


# **Dataset 16:**  
# Conectividad al servicio de Internet  
Nombre del archivo CSV: conectividad_al_servicio_de_internet.csv

In [200]:
conectividad_al_servicio_de_internet = pd.read_csv('./datasets/conectividad_al_servicio_de_internet.csv')

In [201]:
conectividad_al_servicio_de_internet.shape

(4311, 16)

In [202]:
conectividad_al_servicio_de_internet.head()

,Provincia,Partido,Localidad,Población,ADSL,Cablemódem,Dial Up,Fibra óptica,Satelital,Wireless,Telefonía Fija,3G,4G,Link,Latitud,Longitud
0,BUENOS AIRES,9 de Julio,Alfredo Demarchi (Est. Facundo Quiroga),1853,SI,--,--,--,--,--,SI,SI,SI,6588010.0,"-35,29298195","-61,40725421"
1,BUENOS AIRES,9 de Julio,Carlos María Naón,497,--,--,--,--,--,--,SI,SI,SI,6588020.0,"-35,23949938","-60,82513588"
2,BUENOS AIRES,9 de Julio,Dudignac,2670,--,--,SI,--,--,SI,SI,SI,SI,6588040.0,"-35,65119285","-60,70989628"
3,BUENOS AIRES,9 de Julio,La Aurora (Est. La Niña),531,--,--,--,--,--,--,--,--,SI,6588050.0,"-35,40794679","-61,21056682"
4,BUENOS AIRES,9 de Julio,Manuel B. Gonnet (Est. French),748,SI,--,--,SI,--,--,SI,SI,SI,6588060.0,"-35,52014145","-60,9989488"


In [203]:
conectividad_al_servicio_de_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311 entries, 0 to 4310
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Provincia       4311 non-null   object 
 1   Partido         4311 non-null   object 
 2   Localidad       4311 non-null   object 
 3   Población       4311 non-null   int64  
 4   ADSL            4311 non-null   object 
 5   Cablemódem      4311 non-null   object 
 6   Dial Up         4311 non-null   object 
 7   Fibra óptica    4311 non-null   object 
 8   Satelital       4311 non-null   object 
 9   Wireless        4311 non-null   object 
 10  Telefonía Fija  4311 non-null   object 
 11  3G              4311 non-null   object 
 12  4G              4311 non-null   object 
 13  Link            4299 non-null   float64
 14  Latitud         4311 non-null   object 
 15  Longitud        4311 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 539.0+ KB


Después de echar un primer vistazo a los datos, procedemos a guardar el archivo en formato CSV.

In [204]:
# Guardar el DataFrame modificado en un archivo CSV
conectividad_al_servicio_de_internet.to_csv('datasets_cleaned/conectividad_al_servicio_de_internet.csv', index=False)

# **Dataset 17:**  
# Listado de localidades con conectividad a internet.  
Nombre del archivo CSV: listado_de_localidades_con_conectividad_a_internet..csv

In [205]:
listado_de_localidades_con_conectividad = pd.read_csv('./datasets/listado_de_localidades_con_conectividad_a_internet..csv')

In [206]:
listado_de_localidades_con_conectividad.shape

(4311, 12)

In [207]:
listado_de_localidades_con_conectividad.head()

,Provincia,Partido,Localidad,ADSL,Cablemódem,Dial Up,Fibra óptica,4G,3G,Telefonía Fija,Wireless,Satelital
0,BUENOS AIRES,9 de Julio,Alfredo Demarchi (Est. Facundo Quiroga),SI,--,--,--,SI,SI,SI,--,--
1,BUENOS AIRES,9 de Julio,Carlos María Naón,--,--,--,--,SI,SI,SI,--,--
2,BUENOS AIRES,9 de Julio,Dudignac,--,--,SI,--,SI,SI,SI,SI,--
3,BUENOS AIRES,9 de Julio,La Aurora (Est. La Niña),--,--,--,--,SI,--,--,--,--
4,BUENOS AIRES,9 de Julio,Manuel B. Gonnet (Est. French),SI,--,--,SI,SI,SI,SI,--,--


In [208]:
listado_de_localidades_con_conectividad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311 entries, 0 to 4310
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provincia       4311 non-null   object
 1   Partido         4311 non-null   object
 2   Localidad       4311 non-null   object
 3   ADSL            4311 non-null   object
 4   Cablemódem      4311 non-null   object
 5   Dial Up         4311 non-null   object
 6   Fibra óptica    4311 non-null   object
 7   4G              4311 non-null   object
 8   3G              4311 non-null   object
 9   Telefonía Fija  4311 non-null   object
 10  Wireless        4311 non-null   object
 11  Satelital       4311 non-null   object
dtypes: object(12)
memory usage: 404.3+ KB


Después de echar un primer vistazo a los datos y realizar algunos ajustes necesarios, estamos listos para guardar el archivo en formato CSV.

In [209]:
# Guardar el DataFrame modificado en un archivo CSV
listado_de_localidades_con_conectividad.to_csv('datasets_cleaned/listado_de_localidades_con_conectividad.csv', index=False)

# **Dataset 18:**  
# Indicadores macroeconómicos  
Nombre del archivo CSV: indicadores_macroeconómicos.csv

In [210]:
indicadores_macroeconomicos = pd.read_csv('./datasets/indicadores_macroeconómicos.csv')

In [211]:
indicadores_macroeconomicos.shape

(22, 7)

In [212]:
indicadores_macroeconomicos

,Año,Trimestre,IPC US,IPC AR,USDARS oficial,USDARS blue,PBI millones
0,2013,1,232773,298,"501,00","886,00",2.888.981
1,2013,2,233504,309,"524,00","866,00",3.387.811
2,2013,3,234149,331,"558,00","976,00",3.436.547
3,2013,4,233049,351,"605,00","1.143,00",3.679.896
4,2014,1,236293,402,"760,00","1.114,00",3.917.649
5,2014,2,238343,433,"806,00","1.344,00",4.702.630
6,2014,3,238031,461,"830,00","1.378,00",4.685.503
7,2014,4,234812,491,"851,00","1.324,00",5.010.564
8,2015,1,236119,518,"869,00","1.272,00",5.092.694
9,2015,2,238638,549,"896,00","1.506,00",5.951.479


In [213]:
indicadores_macroeconomicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Año             22 non-null     int64 
 1   Trimestre       22 non-null     int64 
 2   IPC US          22 non-null     int64 
 3   IPC AR          22 non-null     int64 
 4   USDARS oficial  22 non-null     object
 5   USDARS blue     22 non-null     object
 6   PBI millones    21 non-null     object
dtypes: int64(4), object(3)
memory usage: 1.3+ KB


Tenemos un valor nulo en la variable `PBI millones`, convertimos en 0 los valores nulos.

In [214]:
indicadores_macroeconomicos.fillna(0, inplace=True)

Llevamos a cabo la transformación de los datos numéricos, normalizando la representación de decimales y miles mediante el reemplazo de comas y puntos. Esta acción es fundamental para garantizar la precisión en los análisis subsiguientes y facilitar una visualización correcta de los datos.

In [215]:
indicadores_macroeconomicos['PBI millones'] = indicadores_macroeconomicos['PBI millones'].apply(lambda x: str(x).replace('.', ''))

In [216]:
# Seleccionar columnas, excluyendo 'USDARS oficial' y 'USDARS blue'
columnas = indicadores_macroeconomicos.columns.difference(['USDARS oficial', 'USDARS blue'])

# Convertir las columnas seleccionadas a tipo de dato entero
indicadores_macroeconomicos[columnas] = indicadores_macroeconomicos[columnas].astype(int)


In [217]:
indicadores_macroeconomicos.head()

,Año,Trimestre,IPC US,IPC AR,USDARS oficial,USDARS blue,PBI millones
0,2013,1,232773,298,"501,00","886,00",2888981
1,2013,2,233504,309,"524,00","866,00",3387811
2,2013,3,234149,331,"558,00","976,00",3436547
3,2013,4,233049,351,"605,00","1.143,00",3679896
4,2014,1,236293,402,"760,00","1.114,00",3917649


Hacemos la transformacion para las columnas `'USDARS oficial'` y `'USDARS blue'` primero sacandole los puntos de las cantidades que expresan miles y luego la coma pasarla a punto para determinar los decimales


In [218]:
indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']] = indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']].applymap(lambda x: str(x).replace('.', ''))

In [219]:
indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']] = indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']].applymap(lambda x: str(x).replace(',', '.'))
indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']] = indicadores_macroeconomicos[['USDARS oficial', 'USDARS blue']].astype(float)


In [220]:
indicadores_macroeconomicos.head()

,Año,Trimestre,IPC US,IPC AR,USDARS oficial,USDARS blue,PBI millones
0,2013,1,232773,298,501.0,886.0,2888981
1,2013,2,233504,309,524.0,866.0,3387811
2,2013,3,234149,331,558.0,976.0,3436547
3,2013,4,233049,351,605.0,1143.0,3679896
4,2014,1,236293,402,760.0,1114.0,3917649


Después de echar un primer vistazo a los datos y realizar algunos ajustes necesarios, estamos listos para guardar el archivo en formato CSV.

In [221]:
# Guardar el DataFrame modificado en un archivo CSV
indicadores_macroeconomicos.to_csv('datasets_cleaned/indicadores_macroeconomicos.csv', index=False)
